# 2. Clustering
In this exercise, we will extract features from metascientific-article titles/abstracts for the purpose of clustering the articles. We will also use the features to try to predict how many citations the articles received. We will make use of a dataset of 1,124 articles obtained using the following query on Scopus: `"science of science" OR "metascience" OR "meta science"`.

## Environment Setup

In [ ]:
import sys
if 'google.colab' in sys.modules:  # If in Google Colab environment
    # Mount google drive to enable access to data files
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Install requisite packages
    !pip install sentence_transformers pacmap &> /dev/null

    # Change working directory
    %cd /content/drive/MyDrive/LLM4SciSci

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from pacmap import PaCMAP
from collections import Counter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

## Feature Extraction
The code begins by loading the data as `pandas.DataFrame` objects and concatenating the titles and abstracts into a single column (`'text'`).

In [ ]:
# Load the data with only the desired columns
data = pd.read_csv('science_of_science.csv', usecols=['title', 'abstract', 'keywords', 'year', 'citations'])

# Concatenate titles and abstracts
data['text'] = data['title'] + './n/n' + data['abstract']
data

It next initializes the `SentenceTransformer` model `'all-MiniLM-L6-v2'` and extracts features from the training data using the `.encode()` method.

In [ ]:
# Initialize feature extraction pipeline
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extract features
features = model.encode(data['text'], show_progress_bar=True)
features

To visualize the article features (and reduce their dimensionality for clustering downstream), the code initializes a `PaCMAP` object and uses it to project the features into 3 components.

In [ ]:
# Initialise PaCMAP with random_state=42 for reproducibility
pacmap_3d = PaCMAP(n_components=3, random_state=42)

# Project features into 3 dimensions
features_3d = pacmap_3d.fit_transform(features)
features_3d

These components can then be visualized using `matplotlib`'s 3-d plotting functionality:

In [ ]:
# Plot features using scatterplot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(features_3d[:, 0], features_3d[:, 1], features_3d[:, 2], alpha=.5)

## Clustering
The code next clusters the features into 5 groups using `sklearn`'s [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html):

In [ ]:
# Cluster using KMeans
kmeans = KMeans(n_clusters=5, random_state=42).fit(features_3d)

# Plot clusters
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(
    features_3d[:, 0], features_3d[:, 1], features_3d[:, 2],    # x, y, z
    c=kmeans.labels_,   # Color with cluster labels
    cmap="viridis", alpha=.8
)
# Rotate the plot
ax.view_init(elev=30, azim=45)  # Adjust angles as needed

**Task 1**: Try adjusting the `elev` (elevation angle), `azim` (horizontal rotation), and `alpha` (opacity, [0,1]) parameters above to get a better intuition for the clusters' 3-dimensional structure.

The code next save the cluster labels as a new column (`'cluster'`) in `data` and saves the results to a new csv file in your Google Drive for reuse in the next exercise.

In [ ]:
# Saving the clusters for 3_text_generation
data['cluster'] = pd.Series(kmeans.labels_, dtype=str)
data.to_csv('science_of_science_clusters.csv', index=False)

In order to give the clusters meaningful labels, we can make use of the `'keywords'` column. Specifically, the code counts the number of times each key word occurs in each cluster, and saves the results to `cluster_keyword_counts`. These counts can be used to characterize the clusters in terms of their most informative keywords.

In [ ]:
cluster_keyword_counts = {} # Dictionary for storing keyword counts in each cluster
unique_keywords = set() # Set for storing all unique keywords

# Iterate through each cluster, count the frequency of each keyword
for cluster in data['cluster'].unique():

    # Select rows for the given cluster
    cluster_data = data.query('cluster == @cluster')

    # Concatenate all keywords into a single string, remove surrounding whitespace, and lowercase
    keywords = cluster_data['keywords'].str.cat(sep='').split(';')
    keywords = [keyword.strip().lower() for keyword in keywords]

    # Save unique keywords
    unique_keywords = unique_keywords.union(set(keywords))

    # Count the frequency of each keyword and save to cluster_keyword_counts
    keyword_counts = Counter(keywords)
    cluster_keyword_counts[cluster] = {keyword.strip().lower(): count for keyword, count in keyword_counts.items()}

# Printing the number of keywords for each cluster
{cluster: len(keywords) for cluster, keywords in cluster_keyword_counts.items()}

The code next reshapes the data in `cluster_keyword_counts` into a `pd.DataFrame` where each row represents a cluster, each column a keyword, and each cell the number of times that keyword occurs in that cluster. It also drops rare words (words occuring <= 3 times) to prevent them from distorting/inflating our pointwise mutual information estimates in the next step.

In [ ]:
# Building cluster-keyword count matrix
count_matrix = pd.DataFrame(index=cluster_keyword_counts.keys(), columns=list(unique_keywords))
for cluster in count_matrix.index:
    for keyword in count_matrix.columns:
        count_matrix.loc[cluster, keyword] = cluster_keyword_counts[cluster].get(keyword, 0)

count_matrix = count_matrix.astype(int) # Convert to int format
count_matrix = count_matrix.loc[:, count_matrix.sum() > 3] # Drop keywords occurring <=3 times
count_matrix

Although we could in-principle already use the `count_matrix` to characterize the clusters, it's likely that high-frequency, less informative keywords end up dominating these characterizations. Instead, the code transform the `count_matrix` by taking the pointwise mutual information (PMI), which ["draws on the intuition that the best way to weigh the association between two words [, or a word and its cluster,] is to ask how much more the two words co-occur in a corpus than we would have expected them to appear by chance."](https://en.wikipedia.org/wiki/Pointwise_mutual_information).

In [ ]:
def compute_pmi(df):

    mat = df.to_numpy(dtype=np.float64)  # Convert DataFrame to NumPy array
    p = mat / np.sum(mat)  # Compute probability matrix

    row_sums = np.sum(p, axis=1, keepdims=True)
    col_sums = np.sum(p, axis=0, keepdims=True) ** 0.7

    expected = row_sums @ col_sums  # Outer product of row and column sums
    pmi = p / expected  # Compute PMI matrix (don't take log since we only care about order in next part)

    return pd.DataFrame(pmi, index=df.index, columns=df.columns)


# Compute pmi
pmi_matrix = compute_pmi(count_matrix)
pmi_matrix

With the PMI estimates computed, the clusters can now be characterized by taking the 5 highest-PMI keywords:

In [ ]:
# Get top 5 highest-PMI keywords
top_keywords = pmi_matrix.apply(lambda row: row.nlargest(5).index.tolist(), axis=1)
top_keywords.to_dict()

The resulting clusters appear relatively coherent and informative. Nevertheless, there may be better ways to characterize the clusters. There may also be situations where we don't have access to a handy set of keywords. Can you think of other ways to characterize the clusters (e.g., using LLMs instead of co-occurence statistics)?

## Prediction

In this section, we will leverage the features (and the rough age of the article) to try to predict the number of citations the articles received. The code begins by converting the features to a `pd.DataFrame` and appending the age of the article as an additional predictor variable.

In [ ]:
features = pd.DataFrame(features)
features['age'] = data['year'].max() - data['year']
features.columns = [str(name) for name in features.columns]
features

The data is next split into a training and test set using `sklearn`'s [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). The features are then standardised before being fed into `RidgeCV`. This is crucial, since `RidgeCV` uses l2 (ridge) regularisation to prevent over-fitting, which assumes that the features have the same scaling. The regression is then trained on the training data using the `fit` method. Note that `RidgeCV` will automatically perform cross-validation on the training data to find the best alpha value from the list of `alphas` provided. Performance on the training data is then evaluated using the `score` method.

In [ ]:
# Splitting data into a training (80%) and test (20%) to evaluate out-of-sample performance
X_train, X_test, y_train, y_test = train_test_split(
    features, np.log1p(data['citations']), test_size=0.2, random_state=42
)

# Standardize features
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

# Initialize classifier
ridge = RidgeCV(alphas=np.logspace(-10, 10, 100))

# Train classifier
ridge.fit(X_train, y_train)
f"Train accuracy: {ridge.score(X_train, y_train):.2f}"

Features are next extracted for the test set and standardised using the same `StandardScaler` object that was fitted on the training data to prevent data leakage. The regression is then evaluated on the test data using the `score` method.

In [ ]:
# Standardising features
X_test = scaler.transform(X_test)

# Test classifier
f"Test accuracy: {ridge.score(X_test, y_test):.2f}"

## General tasks


**TASK 2:** Go to the `SentenceTransformers` [model page](https://sbert.net/docs/sentence_transformer/pretrained_models.html) and find a model of a slightly larger size than  `'all-MiniLM-L6-v2'` (e.g. `all-mpnet-base-v2'`). Open the Hugging Face model card by clicking on the model name and then the link next to 'Model Card'. Replace the `'all-MiniLM-L6-v2` in the code above with the models name (make sure to copy the full name by clicking on the copy-icon to the right of the name at the top of the page). Re-run the entire analysis. Do the article clusters change? Does predictive performance improve?

**TASK 3 (BONUS):** Consider how you might apply the above methods in your own research. If you have dataset in mind, try uploading it to the repository in your Google Drive and loading it into this notebook to run the analysis with your own data (**Caution**: make sure you don't delete the original `science_of_science.csv` dataset, as we will need it for the next exercise).

